In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz
import pytesseract
import io
import sys

from pprint import pprint
from PIL import Image
from pdf2image import convert_from_path
from platform import system
from difflib import get_close_matches

import util
import lines
import group
import label
import records
import extract
import date

In [2]:
test_files_dir = ".././test_files/"
test_dfs_dir = test_files_dir + "tesseract_data_frames/"
test_output_dir = test_files_dir + "extracted_indexes/"
test_files = util.list_files(test_files_dir)
test_dfs = util.list_files(test_dfs_dir)
file = test_dfs[-2]

#indexes_path = "/home/jeusa/Files/Documents/Indexex/ILO_yearly_indices/tesseract_data_frames"
#output_dir = "/home/jeusa/Files/Documents/Indexex/ILO_yearly_indices/extracted_indexes"

if system()=="Windows":
    indexes_path = "H:\Documents\Indexex\ILO_yearly_indices\\tesseract_data_frames"
    output_dir = "H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes"

#indexes_csv = util.list_files(indexes_path, suffix=".csv")

display(test_dfs, file)

['.././test_files/tesseract_data_frames/LS_index_1986_split.csv',
 '.././test_files/tesseract_data_frames/LS_index_1921.csv',
 '.././test_files/tesseract_data_frames/LS_index_1976.csv',
 '.././test_files/tesseract_data_frames/LS_index_1944.csv',
 '.././test_files/tesseract_data_frames/LS_index_1986.csv',
 '.././test_files/tesseract_data_frames/LS_index_1920.csv']

'.././test_files/tesseract_data_frames/LS_index_1986.csv'

In [3]:
#pdf_df = util.ocr(file, start_page=1, save_to=".././test_files")
pdf_df = pd.read_csv(file)
lines_df = lines.make_lines_df_from_ocr(pdf_df)
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df)
borders = lines.make_borders_df(bins_x0, bins_x1)

lines_df

,line_text,artifact_text,x0,y0,x1,y1,page,dx
0,International Legislative Series,,224,236,2041,287,1,1817
1,Labour Office 1987—List No. 3,,225,289,2041,324,1,1816
2,LIST OF RECENT LEGISLATION,,587,404,1680,456,1,1093
3,This list indicates the more important enactme...,,288,561,2044,603,1,1756
4,noted in the official gazettes recently receiv...,,218,608,1729,651,1,1511
...,...,...,...,...,...,...,...,...
4687,(195) 89 1987—List No. 3,,243,264,2059,317,89,1816
4688,Spain—Uruguay,,244,405,561,448,89,317
4689,19/11/1986 Additional international technical ...,,248,474,2066,517,89,1818
4690,matters between the Republic of Uruguay and th...,,311,521,2066,565,89,1755


In [4]:
#rec_df = extract.extract_indexes(pdf_df, save_to=test_output_dir + os.path.basename(file))
rec_df = extract.extract_indexes(pdf_df, file)
rec_df

,country,region,text,date,year,page,extracted_date,extracted_day,extracted_month,extracted_year,full_text
0,ALGERIA,,Decree No. 85-243 laying down model rules for ...,1.10.,1985,1,1/X/1985,1,X,1985,1/X/1985 Decree No. 85-243 laying down model r...
1,ALGERIA,,Decree No. 85-306 to establish a general inspe...,17.12.,1985,1,17/X11/1985,17,X11,1985,17/X11/1985 Decree No. 85-306 to establish a g...
2,ALGERIA,,"National Charter. (ibid., 16 Feb.) The new Nat...",9.2.,1986,1,9/11/1986,9,11,1986,"9/11/1986 National Charter. (ibid., 16 Feb.) T..."
3,ALGERIA,,Decree No. 86-60 determining the provisions ap...,25.3.,1986,1,25/I1I/1986,25,I1I,1986,25/I1I/1986 Decree No. 86-60 determining the p...
4,ALGERIA,,Order concerning the provisions to make in reg...,30.4.,1986,1,30/1V/1986,30,1V,1986,30/1V/1986 Order concerning the provisions to ...
...,...,...,...,...,...,...,...,...,...,...,...
964,EEC,Norway—Switzerland,Agreement to supplement the Administrative Arr...,28.6.,1985,88,28/V1/1985,28,V1,1985,28/V1/1985 Agreement to supplement the Adminis...
965,EEC,Portugal—Spain,Agreement between Spain and Portugal regarding...,12.11.,1983,88,12/X1/1983,12,X1,1983,12/X1/1983 Agreement between Spain and Portuga...
966,EEC,Senegal—Zaire,Co-operation agreement between the Government ...,26.3.,1986,88,26/11I/1986,26,11I,1986,26/11I/1986 Co-operation agreement between the...
967,EEC,Senegal—Zaire,(195) 89 1987—List No. 3,,,89,,,,,(195) 89 1987—List No. 3


In [32]:
rec_df.loc[rec_df["page"]==11]

,country,region,text,date,year,page,extracted_date,extracted_day,extracted_month,extracted_year,full_text
102,BELGIUM,,(117) 1] 1987—List No. 3,,,11,,,,,(117) 1] 1987—List No. 3
103,BELGIUM,,Royal Order amending the Royal Order of 10 Ja...,15.10.,1986,11,15/X/1986,15,X,1986,15/X/1986 Royal Order amending the Royal Order...
104,BELIZE,,Labour (Amendment) Act 1986. No. 11. Amends t...,27.3.,1986,11,27/111/1986,27,111,1986,27/111/1986 Labour (Amendment) Act 1986. No. 1...
105,BRAZIL,,Act No. 7414 to amend s. 135 of the Consolida...,9.12.,1985,11,9/XII/1985,9,XII,1985,9/XII/1985 Act No. 7414 to amend s. 135 of the...
106,BRAZIL,,"Act No. 7415, to amend Act. No. 605 of 5 Janu...",9.12.,1985,11,9/X11/1985,9,X11,1985,"9/X11/1985 Act No. 7415, to amend Act. No. 605..."
107,BRAZIL,,Act No. 7430 to amend the wording of the intr...,17.12.,1985,11,17/X11/1985,17,X11,1985,17/X11/1985 Act No. 7430 to amend the wording ...
108,BRAZIL,,Decree No. 92366 to amend para. III of s. 25 ...,4.2.,1986,11,4/11/1986,4,11,1986,4/11/1986 Decree No. 92366 to amend para. III ...
109,BRAZIL,,Legislative Decree No. 2283 to make provision...,27.2.,1986,11,27/11/1986,27,11,1986,27/11/1986 Legislative Decree No. 2283 to make...
110,BRAZIL,,Legislative Decree No. 2284 to maintain a new...,10.3.,1986,11,10/111/1986,10,111,1986,10/111/1986 Legislative Decree No. 2284 to mai...


In [34]:
a = rec_df.loc[230]["text"]
a

' Decree No. 1501-A, to reform the general regulations governing the Education Act. (ibid., 28 Jan.) '

In [35]:
def clean_text(rec_df):
    df = rec_df.copy()
    
    reg_w = " {2,}"
    reg_p = "( *[\.;,+] *)+$"
    
    for i, row in df.iterrows():
        t = row["text"]
        t = re.sub(reg_p, "", t)
        t = re.sub(reg_w, " ", t)
        t = t.strip()
        df.loc[i, "text"] = t
    
    return df

In [15]:
lines_df.loc[lines_df["page"]==9]

,line_text,artifact_text,x0,y0,x1,y1,page,dx
428,(115) 9 1987—List No. 3,,212,248,2031,300,9,1819
429,BARBADOS (cont.),,209,389,601,429,9,392
430,"Make provisions, inter alia, for survivor’s pe...",[,291,455,1300,508,9,1009
431,25/VII/1985 National Insurance and Social Secu...,,209,527,2027,573,9,1818
432,Regulations 1985. S.I. 1985 No. 124.,,277,575,975,616,9,698
433,Contains provisions for death benefits. ],[,289,639,1051,691,9,762
434,30/X11/1985 Sugar Industry (Sugar Workers Prov...,,209,711,2019,755,9,1810
435,Provides for the imposition of a levy on sugar...,[,286,780,2026,827,9,1740
436,paid to the National Insurance Board. ],,273,823,1029,876,9,756
437,"27/I11/1986, Factories (Amendment) Act 1986. N...",,211,897,1251,939,9,1040


In [8]:
d = lines_df.loc[lines_df["page"]==9]
d_x0 = group_lines(d, "x1")
d_x0["page"] = 9
d_x0

,x1,lines,last_x1,last_x1_mean,count,page
0,886,364,886,886,1,9
1,1453,363,1453,1453,1,9
2,528,359,528,528,1,9
3,"[1761, 1755]","[356, 368]","[1761, 1755]",1758.0,2,9
4,"[1629, 1629]","[352, 369]","[1629, 1629]",1629.0,2,9
5,974,349,974,974,1,9
6,1247,338,1247,1247,1,9
7,421,335,421,421,1,9
8,"[1702, 1699]","[334, 367]","[1702, 1699]",1700.5,2,9
9,"[353, 356]","[330, 353]","[353, 356]",354.5,2,9


In [20]:
r_x0 = get_relevant_x0_bins(d_x0, 3)
r_x0

,x0,lines,last_x0,last_x0_mean,count,page
4,"[92, 112, 89, 117, 90, 92, 90, 104, 94, 94, 108]","[324, 326, 330, 331, 335, 351, 353, 365, 366, ...","[94, 108]",101.0,11,9
3,"[137, 136, 134, 134, 136, 137, 149, 138, 138, ...","[325, 327, 336, 339, 345, 350, 354, 357, 360, ...","[120, 136]",128.0,11,9
2,"[391, 389, 385, 391, 391, 389, 389, 392, 391, ...","[328, 329, 332, 333, 334, 337, 338, 340, 341, ...","[393, 389]",391.0,23,9


In [21]:
lines_df.loc[r_x0.iloc[0]["lines"]]

,line_text,artifact_text,x0,y0,x1,y1,page,dx
324,NORWAY (continued),,92,400,692,455,9,600
326,"i old age pensions’. .(N.L. Part I, p. 408; Pa...",,112,534,1663,597,9,1551
330,PANAMA,,89,796,353,840,9,264
331,"Aug, 22 L.D. No. 55, to make an addition to Pa...",,117,900,1833,943,9,1716
335,PORTUGAL,,90,1115,421,1169,9,331
351,"Tes ‘of 25th May, 1931, to set up the National...",,92,2092,1831,2160,9,1739
353,SWEDEN,,90,2248,356,2305,9,266
365,L.S. 1936 (Nor. 4); amendments: 1937 (Nor. 5);...,*,104,3008,1837,3093,9,1733
366,1930 (Nor. 2); amendments: 1932 (Nor. 4); 1933...,,94,3065,1834,3132,9,1740
367,(Nor. 2); 1937 (Nor. 1); 1941 (Nor. 1); 1942 (...,,94,3116,1699,3180,9,1605


In [7]:
df = label.assign_types(lines_df, bins_x0, bins_x1, x0_n)
df = label.assign_labels(df, x0_n)

co_df, p_l, p_g = label.correct_x0_types(df, bins_x0, bins_x1, x0_n)
ind_df = label.assign_labels(co_df, x0_n)
ind_df = label.approve_correction(df, ind_df, p_l)

ind_df["new_label"] = ind_df["label"]
ind_df = label.improve_country_classification(ind_df)

ind_df = records.extract_records(ind_df)

In [22]:
df.loc[df["page"]==9]

,line_text,artifact_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
323,g — 1944—-Indexes I,—,898,313,1839,347,9,941,-1,2,other
324,NORWAY (continued),,92,400,692,455,9,600,0,0,country
325,Aug. 24 Act to amend provisionally the Act of ...,,137,489,1835,560,9,1698,1,2,start
326,"i old age pensions’. .(N.L. Part I, p. 408; Pa...",,112,534,1663,597,9,1551,0,1,country
327,Oct. 19 Act to amend [the first paragraph of s...,,136,610,1834,665,9,1698,1,2,start
328,"April, 1942, to supplement the Act of 6th June...",,391,664,1833,744,9,1442,2,2,middle
329,"sickness insurance”. (N.L. Part I, pp. 475-476...",,389,704,1827,759,9,1438,2,2,middle
330,PANAMA,,89,796,353,840,9,264,0,0,country
331,"Aug, 22 L.D. No. 55, to make an addition to Pa...",,117,900,1833,943,9,1716,0,2,other
332,"27th April, 1943, respecting social insurance“...",,385,946,1835,992,9,1450,2,2,middle


In [23]:
co_df.loc[co_df["page"]==9]

,line_text,artifact_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
323,g — 1944—-Indexes I,—,898,313,1839,347,9,941,-1,2,other
324,NORWAY (continued),,92,400,692,455,9,600,1,0,country
325,Aug. 24 Act to amend provisionally the Act of ...,,137,489,1835,560,9,1698,2,2,start
326,"i old age pensions’. .(N.L. Part I, p. 408; Pa...",,112,534,1663,597,9,1551,1,1,country
327,Oct. 19 Act to amend [the first paragraph of s...,,136,610,1834,665,9,1698,2,2,start
328,"April, 1942, to supplement the Act of 6th June...",,391,664,1833,744,9,1442,3,2,middle
329,"sickness insurance”. (N.L. Part I, pp. 475-476...",,389,704,1827,759,9,1438,3,2,middle
330,PANAMA,,89,796,353,840,9,264,1,0,country
331,"Aug, 22 L.D. No. 55, to make an addition to Pa...",,117,900,1833,943,9,1716,1,2,other
332,"27th April, 1943, respecting social insurance“...",,385,946,1835,992,9,1450,3,2,middle


In [26]:
co, p_l, p_g = correct_x0_types(df, bins_x0, bins_x1, x0_n)

Width median 1752.2894736842106
[(1658.9333333333332, 2), (1775.093023255814, 14), (1703.0959302325582, 15), (1709.0063795853268, 25), (1707.2626538987688, 39), (1828.5263157894738, 64), (1705.3108974358975, 69), (1726.8848484848486, 70), (1703.9268292682927, 77), (1728.189024390244, 78), (1702.4452690166975, 79), (1732.2894736842106, 81)]
Pages small width [2, 15, 25, 39, 69, 70, 77, 78, 79, 81]
Pages big width [14, 64]


In [28]:
co.loc[co["page"]==9]

,line_text,artifact_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
323,g — 1944—-Indexes I,—,898,313,1839,347,9,941,-1,2,other
324,NORWAY (continued),,92,400,692,455,9,600,0,0,country
325,Aug. 24 Act to amend provisionally the Act of ...,,137,489,1835,560,9,1698,1,2,start
326,"i old age pensions’. .(N.L. Part I, p. 408; Pa...",,112,534,1663,597,9,1551,0,1,country
327,Oct. 19 Act to amend [the first paragraph of s...,,136,610,1834,665,9,1698,1,2,start
328,"April, 1942, to supplement the Act of 6th June...",,391,664,1833,744,9,1442,2,2,middle
329,"sickness insurance”. (N.L. Part I, pp. 475-476...",,389,704,1827,759,9,1438,2,2,middle
330,PANAMA,,89,796,353,840,9,264,0,0,country
331,"Aug, 22 L.D. No. 55, to make an addition to Pa...",,117,900,1833,943,9,1716,0,2,other
332,"27th April, 1943, respecting social insurance“...",,385,946,1835,992,9,1450,2,2,middle


In [9]:
ind_df.loc[ind_df["page"]==10]

,country,region,text,page
90,SWEDEN,,"on _ workers’ protection™, . (S.F,-p, 259)» - ...",10
91,SWEDEN,,"1931, respecting sick funds® <r (S.F. pp. 444-...",10
92,SWEDEN,,SWITZERLAND CONFEDERATION Jan. 400 Resolution ...,10
93,SWEDEN,,"1935 (Swe. 4); 1936 (Swe. 7, A); see also: 193...",10
94,SWEDEN,,"(Swe. 2, C). * Consolidated text: L.S. 1931 (S...",10
95,SWEDEN,,"(Swe. 2); 1937 (Swe. 3); 1938 (Swe. 4). — *""L....",10
96,SWEDEN,,"1 S, 1930 (Swe. 1); amendments: L.S. 1938 (Swe...",10
97,SWEDEN,,"1942 (Swe. 2, B). — “LS. 1940 (Swe. 1, A). 1S....",10
98,SWEDEN,,Bulletin of the International Labour Office (B...,10
99,SWEDEN,,"269-283, and Vol. XIV, 1919, pp. 205-209. — “L...",10


In [25]:
# correct x0_type for pages where something went wrong
def correct_x0_types(lines_df, bins_x0, bins_x1, x0_n):
    df = lines_df.copy()
    start_page = lines_df["page"].min()

    text_widths = list(lines.make_borders_df(bins_x0, bins_x1)["dx"]) # difference between mean of first and last bin for x0 for every page

    tw = text_widths.copy()
    tw.sort()
    width_median = tw[int(len(tw)/2)]
    print("Width median", width_median)

    f = filter(lambda w: (not util.similar_to(w, width_median, 20)), text_widths) # filter widths that differ from the rest
    f = list(f)

    p = []
    for w in range(len(f)):
        p.append((f[w], text_widths.index(f[w]) + start_page)) # add page to strange width
    
    print(p)

    p_l = [a for w, a in p if w<width_median]
    p_g = [a for w, a in p if w>width_median]
    
    print("Pages small width", p_l)
    print("Pages big width", p_g)

    df.loc[df["page"].isin(p_l) & (df["x0_type"]>=0), "x0_type"] +=1 # correct wrong x0_type for p_l

    # correct wrong x0_type (and x1_type) for p_g
    bins = group.get_line_start_end_bins(df.loc[df["page"].isin(p_g)])
    bins_x0_cor = group.get_relevant_x0_bins(bins[0], x0_n, drop_first=True) # drop bin on the very left
    df_cor = label.assign_types(df.loc[df["page"].isin(p_g)], bins_x0_cor, bins_x1.loc[bins_x1["page"].isin(p_g)], x0_n)
    df.loc[df["page"].isin(p_g), ["x0_type", "x1_type"]] = df_cor[["x0_type", "x1_type"]]

    return df, p_l, p_g

In [17]:
for i, file in enumerate(indexes_csv):
    save_path = output_dir + os.sep + os.path.basename(file)
    extract.extract_indexes(pd.read_csv(file), save_to=save_path)

Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1976.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1977.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1978.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1979.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1981.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1982.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_Index_1983.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1983_Spanish.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1984.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_year

## Probleme
- Dokumente mit 2 Spalten FIXED
- Dokumente mit Länder-Überschrift über ganze Breite der Seite IMPROVED
- Länder- und Regionenspalte vereinen?
    - Regionenerkennung oft nicht richtig, weil Groß- und Kleinbuchstaben vom OCR nicht richtig erkannt werden
- Dokumente/Seiten mit vielen Artifakten
    - 1944
        - S. 8-9, schlechte Ländererkennung FIXED
            - S. 25 falsches Labeling FIXED
        - S. 28 Inhalt fehlt? TESSERACT PROBLEM
        - S. 10-11, schlechte Ländererkennung FIXED
- Date-Type-Erkennung fehlerhaft
    - 1920 FIXED
- 1920 Seite 11 Uruguay
    - unvollständige Datums-Erkennung
    

## Todo

